In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
from PIL import Image, ImageTk
import numpy as np
import pandas as pd
import joblib
import xgboost as xgb
import random

# 关键修正：确保模型和标准化器路径与训练时保存的完全一致
model_path = "xgboost_model.json"
scaler_path = "scaler.pkl"
data_path = "testing data.csv"#testing data

# 加载模型和标准化器（保持原逻辑，仅修正路径）
try:
    best_model = xgb.XGBRegressor()
    best_model.load_model(model_path)
    scaler = joblib.load(scaler_path)
except FileNotFoundError as e:
    messagebox.showerror("文件缺失", f"找不到以下文件：\n{str(e)}\n请检查路径是否正确！")
    exit()

data = pd.read_csv(data_path)

# 替换为ML9.csv的新参数（顺序与数据一致）
tk_feature_names = ['b', 'd', 'fc', 'a/d', 'df', 'lf', 'pf', 'ps', 'fy']

# 新参数对应的单位
unit_dict = {
    'b': 'mm',
    'd': 'mm',
    'fc': 'MPa',
    'a/d': '',
    'df': 'mm',
    'lf': 'mm',
    'pf': '%',
    'ps': '%',
    'fy': 'MPa'
}

def fill_random_values():
    random_row = data.sample(n=1).iloc[0]
    for i, entry in enumerate(entries):
        entry.delete(0, tk.END)
        val = random_row[tk_feature_names[i]]
        entry.insert(0, str(int(val)) if val.is_integer() else str(val))

def validate_input():
    try:
        user_input = [float(entry.get()) for entry in entries]
        # 仅保留非负校验，删除所有其他输入限制
        if any(val < 0 for val in user_input):
            raise ValueError("所有输入参数不能为负数！")
        return user_input
    except Exception as e:
        messagebox.showerror("输入错误", str(e))
        return None

def predict():
    user_input = validate_input()
    if user_input is not None:
        user_input_array = np.array(user_input).reshape(1, -1)
        scaled_input = scaler.transform(user_input_array)
        model_prediction = best_model.predict(scaled_input)[0]
        # 去掉公式预测，仅显示XGBoost模型结果
        result_label.config(text=f"V_pred : {model_prediction:.2f} kN", foreground="blue")

root = tk.Tk()
root.title("Shear Capacity of UHPFRC Beams without Stirrups")
root.geometry("1300x850")
root.resizable(False, False)

biaoti_img = Image.open("title.png").resize((980, 110))
biaoti_photo = ImageTk.PhotoImage(biaoti_img)
tk.Label(root, image=biaoti_photo).pack(pady=(30, 0))

frame = tk.Frame(root)
frame.pack(fill="both", expand=True)

left_frame = tk.Frame(frame)
left_frame.pack(side="left", padx=20, pady=20)

tk.Label(left_frame, text="Input parameters:", font=("Arial", 20, "bold")).pack()

random_button = ttk.Button(left_frame, text="Random parameters", command=fill_random_values)
random_button.pack(pady=10)

entries = []
for feature in tk_feature_names:
    row_frame = tk.Frame(left_frame)
    row_frame.pack(fill="x", pady=10)
    
    unit = unit_dict[feature]
    label_text = feature if unit == '' else f"{feature} ({unit})"
    label = tk.Label(row_frame, text=label_text, width=8, font=("Times New Roman", 14, "italic"), anchor="w")
    label.pack(side="left", padx=10)
    
    entry = ttk.Entry(row_frame, font=("Arial", 10), width=10)
    entry.pack(side="left", fill="x", expand=True, padx=3)
    
    entries.append(entry)

fill_random_values()

predict_button = ttk.Button(left_frame, text="predict", command=predict)
predict_button.pack(pady=10)

result_label = tk.Label(left_frame, text="", font=("Arial", 12))
result_label.pack()

shiyi_img = Image.open("parameter meanings.png").resize((950, 630))
shiyi_photo = ImageTk.PhotoImage(shiyi_img)
tk.Label(frame, image=shiyi_photo).pack(side="right", padx=30)

footer_label = tk.Label(root, text="This GUI is developed by College of Civil Engineering of Nanjing Forestry University (k.zheng@njfu.edu.cn)", 
                        font=("Helvetica", 14), fg="darkblue")
footer_label.pack(side="bottom", pady=5)

root.mainloop()